<a href="https://colab.research.google.com/github/weavermonkey/AEMOnDocker/blob/master/captcha_solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import pandas as pd
import numpy as np
import cv2
import glob
import imutils
from imutils import paths
import os
import os.path

In [0]:
SOLVED_CAPTCHA_FOLDER = "images"
OUTPUT_FOLDER = "extracted_letters"
solved_captchas = glob.glob(os.path.join(SOLVED_CAPTCHA_FOLDER, "*"))
counts = {}
for (i, captcha) in enumerate(solved_captchas):
    filename = os.path.basename(captcha)
    captcha_text = os.path.splitext(filename)[0]
    image = cv2.imread(captcha)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.copyMakeBorder(gray, 8, 8, 8, 8, cv2.BORDER_REPLICATE)
    thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV, cv2.THRESH_OTSU)[1]
    contours, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    letter_image_regions = []

    for contour in contours:
      (x, y, w, h) = cv2.boundingRect(contour)
      if w / h > 1.25:
        half_width = int(w / 2)
        letter_image_regions.append((x, y, half_width, h))
        letter_image_regions.append((x + half_width, y, half_width, h))
      else:
        letter_image_regions.append((x, y, w, h))
    letter_image_regions = sorted(letter_image_regions, key=lambda x: x[0])

    for letter_bounding_box, letter_text in zip(letter_image_regions, captcha_text):
      x, y, w, h = letter_bounding_box
      letter_image = gray[y - 2:y + h + 2, x - 2:x + w + 2]
      save_path = os.path.join(OUTPUT_FOLDER, letter_text)
      if not os.path.exists(save_path):
        os.makedirs(save_path)
      count = counts.get(letter_text, 1)
      p = os.path.join(save_path, "{}.png".format(str(count)))
      cv2.imwrite(p, letter_image)
      counts[letter_text] = count + 1

error: ignored

In [0]:
letter_folder = 'extracted_letters'
data = []
labels = []
for image in paths.list_images(letter_folder):
    img = cv2.imread(image)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, (30,30))
    img = np.expand_dims(img, axis = 2)
    label = image.split(os.path.sep)[-2]
    data.append(img)
    labels.append(label)
data = np.array(data, dtype = "float")
labels = np.array(labels)
print(data.shape, labels.shape)

(83, 30, 30, 1) (83,)


In [0]:
data = data/255.0


In [0]:
from sklearn.model_selection import train_test_split
(train_x, val_x, train_y, val_y) = train_test_split(data, labels, test_size=0.2, random_state=0)
print(train_x.shape, val_x.shape, train_y.shape, val_y.shape)


(66, 30, 30, 1) (17, 30, 30, 1) (66,) (17,)


In [0]:
from sklearn.preprocessing import LabelBinarizer
import pickle
lb = LabelBinarizer().fit(train_y)
train_y = lb.transform(train_y)
val_y = lb.transform(val_y)

bin = pickle.dumps(lb)
with open("captcha_labels.pickle", "wb") as f:
    pickle.dump(lb, f)

print(train_y.shape, val_y.shape)

(66, 18) (17, 18)


In [0]:
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Flatten, Dense, Dropout
from keras.callbacks import EarlyStopping


In [0]:
model = Sequential()
model.add(Conv2D(20, (5, 5), padding="same", input_shape=(30, 30, 1), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(50, (5, 5), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(18, activation="softmax"))

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 30, 30, 20)        520       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 15, 15, 20)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 15, 15, 50)        25050     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 7, 7, 50)          0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 2450)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 256)               627456    
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)              

In [0]:
estop = EarlyStopping(patience=10, mode='min', min_delta=0.001, monitor='val_loss')

In [0]:
model.fit(train_x, train_y, validation_data=(val_x, val_y), batch_size=32, epochs=50, verbose=1, callbacks = [estop])

Train on 66 samples, validate on 17 samples
Epoch 1/50
66/66 [==============================] - 0s 5ms/step - loss: 111.1150 - accuracy: 0.0606 - val_loss: 47.1671 - val_accuracy: 0.1765
Epoch 2/50
66/66 [==============================] - 0s 3ms/step - loss: 68.2238 - accuracy: 0.0606 - val_loss: 26.6556 - val_accuracy: 0.0588
Epoch 3/50
66/66 [==============================] - 0s 3ms/step - loss: 24.9162 - accuracy: 0.1515 - val_loss: 8.2834 - val_accuracy: 0.0588
Epoch 4/50
66/66 [==============================] - 0s 3ms/step - loss: 6.3091 - accuracy: 0.1667 - val_loss: 3.9204 - val_accuracy: 0.1765
Epoch 5/50
66/66 [==============================] - 0s 3ms/step - loss: 2.9077 - accuracy: 0.3182 - val_loss: 3.1487 - val_accuracy: 0.1765
Epoch 6/50
66/66 [==============================] - 0s 3ms/step - loss: 2.2159 - accuracy: 0.3788 - val_loss: 2.7641 - val_accuracy: 0.1765
Epoch 7/50
66/66 [==============================] - 0s 3ms/step - loss: 1.8247 - accuracy: 0.3788 - val_loss: 

In [0]:
print(val_y.shape)

(17, 18)


In [0]:
import matplotlib.pyplot as plt
%matplotlib inline
image = cv2.imread('/content/images/PPAJAR.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.copyMakeBorder(gray, 8, 8, 8, 8, cv2.BORDER_REPLICATE)
thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV, cv2.THRESH_OTSU)[1]
contours, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
letter_image_regions = []
for contour in contours:
    # Get the rectangle that contains the contour
    (x, y, w, h) = cv2.boundingRect(contour)
    
    # checking if any counter is too wide
    # if countour is too wide then there could be two letters joined together or are very close to each other
    if w / h > 1.25:
        # Split it in half into two letter regions
        half_width = int(w / 2)
        letter_image_regions.append((x, y, half_width, h))
        letter_image_regions.append((x + half_width, y, half_width, h))
    else:
        letter_image_regions.append((x, y, w, h))
            

# Sort the detected letter images based on the x coordinate to make sure
# we get them from left-to-right so that we match the right image with the right letter  

letter_image_regions = sorted(letter_image_regions, key=lambda x: x[0])

# Create an output image and a list to hold our predicted letters
output = cv2.merge([gray] * 3)
predictions = []
    
# Creating an empty list for storing predicted letters
predictions = []
    
# Save out each letter as a single image
for letter_bounding_box in letter_image_regions:
    # Grab the coordinates of the letter in the image
    x, y, w, h = letter_bounding_box

    # Extract the letter from the original image with a 2-pixel margin around the edge
    letter_image = gray[y - 2:y + h + 2, x - 2:x + w + 2]

    letter_image = cv2.resize(letter_image, (30,30))
        
    # Turn the single image into a 4d list of images
    letter_image = np.expand_dims(letter_image, axis=2)
    letter_image = np.expand_dims(letter_image, axis=0)

    # making prediction
    pred = model.predict(letter_image)
        
    # Convert the one-hot-encoded prediction back to a normal letter
    letter = lb.inverse_transform(pred)[0]
    #predictions.append(letter)
    print((letter))

'''

    # draw the prediction on the output image
    try:
      cv2.rectangle(output, (x - 2, y - 2), (x + w + 4, y + h + 4), (0, 255, 0), 1)
      cv2.putText(output, letter, (x - 5, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.55, (0, 255, 0), 2)
    except Exception as e:
      pass

#captcha_text = "".join(predictions)
#print("CAPTCHA text is: {}".format(captcha_text))
plt.imshow(output)
'''

[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


'\n\n    # draw the prediction on the output image\n    try:\n      cv2.rectangle(output, (x - 2, y - 2), (x + w + 4, y + h + 4), (0, 255, 0), 1)\n      cv2.putText(output, letter, (x - 5, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.55, (0, 255, 0), 2)\n    except Exception as e:\n      pass\n\n#captcha_text = "".join(predictions)\n#print("CAPTCHA text is: {}".format(captcha_text))\nplt.imshow(output)\n'